
# ДЗ 8. ML для бизнеса

__1.__ [Скачать любой понравившийся вам датасет с UCI repository(можете даже свой какой-то взять) и сразу же разделите его на тренировочную и тестовую выборки. Тестовую выборку сохраните на диск и больше не трогайте.](#task01)

__2.__ [Провести EDA (exploratory data analysis: histograms, boxplots - visualization, etc). Например](#task02)

__3.__ [Сделать feature engineering. При этом постарайтесь «завернуть» ваш код для преобразования данных в классы или функции (пример Transformer как в ноутбуке).](#task03)

__4.__ [Обучить базовую модель (любую - на ваш выбор). При этом нужно использовать cross validation](#task04)

__5.__ [Посчитайте разные метрики качества на cross-validation, отрисуйте графики (roc auc)](#task05)

__6.__ [Сделать подбор гиперпараметров с помощью той же самой cross-validation.](#task06)

__7.__ [Сохраните вашу модель на диск](#task07)

__8.__ [(в отдельном ноутбуке!) Загрузите ваш отложенный на шаге 1 датасет и сохраненные ранее файлы и сделайте прогноз моделью.](#task08)

__9.__ [Посчитайте метрики качества и сделайте выводы о том, насколько прогнозы (качество) на «новых» данных отличаются от того, что вы видели на cross-validation. Если будут очень серьезные отличия, то подумайте, где вы ошиблись и как это можно поправить.](#task09)

__10.__ [Бонусный вопрос: у вас есть на выбор два варианта разделения вашего исходного датасета - на train/test и на train/test/val. Подумайте, в каких ситуациях предпочтительнее один или второй вариант либо всегда лучше какой-то один? Какой бы выбрали вы?](#task10)

По методам отбора признаков ссылки:
1. пакет boruta (shadow attributes) - https://github.com/scikit-learn-contrib/boruta_py Там же ссылка на оригинальную статью
2. мега-статья с разбором множества методов отбора признаков (будет круто изучить хотя бы поверхностно) - https://arxiv.org/pdf/1601.07996.pdf

In [1]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

/Users/artem/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from random import randint

## Задание 1

Скачать любой понравившийся вам датасет с UCI repository(можете даже свой какой-то взять) и сразу же разделите его на тренировочную и тестовую выборки. Тестовую выборку сохраните на диск и больше не трогайте.

Загрузим датасет по ссылке: https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv

In [3]:
# ссылка впоследствии перестала работать, поэтому подгружать датасет впоследствии пришлось из файла
# try:
#     path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00468/online_shoppers_intention.csv'
#     print('dataset from link')
# except:
#     path = 'online_shoppers_intention.csv'
#     print('dataset from file')

path = 'online_shoppers_intention.csv'
data = pd.read_csv(path, sep=',')

Обзорный взгляд на датасет:

In [4]:
data.head(3)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.0,0.2,0.2,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.0,0.0,0.1,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.0,0.2,0.2,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False


In [5]:
data.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157213,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


Создавать отложенную выборку на этом этапе пока нет смысла, т.к. мы даже не знаем, какие преобразования признаков потребуются, и лучше их сразу сделать на всё датасете, а уже потом разделять датасет

In [6]:
# таргет изменим на формат "1"/"0" вместо формата True/False
data['Revenue'] = data['Revenue'].replace({True:1, False:0})

В дальнейшем была обнаружена ошибка при использовании пайплайнов (которая съела много времени), заключающаяся в том, что в тестовую выборку могли не попасть те значения, из которых потом будут создаваться новые дамми-признаки. Например, если признак "x" может принимать значения "1", "2", "3", то по итогам применения пайплайнов к трейн-выборке, происходит генерация признаков "x_1", "x_2", "x_3", но если в тестовой выборке признак "x" будет принимать только одно значение (например, "1") , то будет создан только один признак "x_1".

В частности, обнаружилось, что признак "Browser" имеет такие значения, что при некоторых разбивках, не все значения попадают в тестовую выборку, поэтому только ради того, чтобы выполнить ДЗ, пришлось применить очень кривые костыли, а по-хорошему, с этим, конечно, надо разбираться по-нормальному.

Итак, я вынужден удалить все значения признаков, которые встречаются реже чем 20 на датасет. Определим такие признаки:

In [7]:
data['Browser'].value_counts()

2     7961
1     2462
4      736
5      467
6      174
10     163
8      135
3      105
13      61
7       49
12      10
11       6
9        1
Name: Browser, dtype: int64

* значения признака "Browser" равное 12, 11, 9 удалить из датафрейма

In [8]:
df_to_drop1 = data.loc[(data['Browser']==9) |(data['Browser']==11) | (data['Browser']==12)]
list(df_to_drop1.index) #list(data_top.index)

[284,
 3050,
 6236,
 6316,
 7969,
 7980,
 8185,
 8749,
 9025,
 9184,
 9523,
 10531,
 10556,
 10617,
 11188,
 12130,
 12314]

In [9]:
data.drop(list(df_to_drop1.index), inplace=True)

In [10]:
data['TrafficType'].value_counts()

2     3909
1     2445
3     2049
4     1069
13     738
10     450
6      443
8      342
5      260
11     246
20     198
9       42
7       40
15      38
19      17
14      12
18      10
16       3
12       1
17       1
Name: TrafficType, dtype: int64

In [11]:
df_to_drop2 = data.loc[(data['TrafficType']==19) |(data['TrafficType']==14) | (data['TrafficType']==18)|
                       (data['TrafficType']==16)|(data['TrafficType']==12)|(data['TrafficType']==17)]
list(df_to_drop2.index) #list(data_top.index)

[219,
 462,
 518,
 2123,
 2211,
 2424,
 2575,
 2640,
 2674,
 2683,
 2695,
 2699,
 2703,
 2731,
 2775,
 2817,
 2834,
 2848,
 2855,
 3160,
 3581,
 3602,
 3665,
 3748,
 3834,
 3850,
 3949,
 4186,
 4253,
 4288,
 4338,
 4387,
 4394,
 4480,
 4483,
 4554,
 4643,
 4858,
 5077,
 5152,
 5186,
 5225,
 5296,
 7350]

In [12]:
data.drop(list(df_to_drop2.index), inplace=True)

In [13]:
data['TrafficType'].value_counts()

2     3909
1     2445
3     2049
4     1069
13     738
10     450
6      443
8      342
5      260
11     246
20     198
9       42
7       40
15      38
Name: TrafficType, dtype: int64

In [14]:
data['OperatingSystems'].value_counts()

2    6572
1    2581
3    2534
4     477
8      79
6      19
7       7
Name: OperatingSystems, dtype: int64

In [15]:
df_to_drop3 = data.loc[(data['OperatingSystems']==6) |(data['OperatingSystems']==7)]
list(df_to_drop3.index) #list(data_top.index)

[260,
 883,
 2218,
 2414,
 2680,
 3207,
 3446,
 3458,
 4249,
 4531,
 5024,
 5128,
 5208,
 5377,
 5440,
 5561,
 7103,
 7244,
 7492,
 7772,
 9131,
 10316,
 10413,
 11082,
 11540,
 11996]

In [16]:
data.drop(list(df_to_drop3.index), inplace=True)

### Creating a holdout dataset (test)

Создадим отложенную выборку, которую мы не будем трогать до самого конца.

In [17]:
from sklearn.model_selection import train_test_split

target = 'Revenue'

x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=target), data[target], test_size=0.33, random_state=42)

In [18]:
# # признаки
# x_train = data.iloc[:7500,:-1]
# x_test = data.iloc[7500:10000,:-1]
# x_valid = data.iloc[10000:, :-1]

In [19]:
# # таргеты
# y_train = data.iloc[:7500,-1]
# y_test = data.iloc[7500:10000,-1]
# y_valid = data.iloc[10000:,-1]

In [20]:
x_test.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend
10109,0,0.0,0,0.0,6,123.750000,0.000000,0.011111,0.000000,0.0,Dec,2,2,7,6,Returning_Visitor,False
10030,0,0.0,0,0.0,8,101.833333,0.000000,0.058333,0.000000,0.0,Nov,2,2,3,2,Returning_Visitor,False
3236,6,77.0,0,0.0,42,3436.066667,0.000758,0.019481,0.000000,0.0,May,2,2,3,4,Returning_Visitor,False
9793,1,65.5,1,0.0,82,5316.459524,0.005000,0.011250,77.259517,0.0,Nov,1,1,9,3,Returning_Visitor,True
9000,0,0.0,0,0.0,2,0.000000,0.200000,0.200000,0.000000,0.0,Nov,2,2,9,13,Returning_Visitor,False


Сохраняем тестовую и валидационную выборки в файлы

In [21]:
x_test.to_csv('test.csv', sep=";", index=None)
# x_valid.to_csv('valid.csv', sep=";", index=None)
x_train.to_csv('train.csv', sep=";", index=None)
y_test.to_csv('y_test.csv', sep=";", index=None)
# y_valid.to_csv('y_valid.csv', sep=";", index=None)
y_train.to_csv('y_train.csv', sep=";", index=None)

In [22]:
df = data.copy() # первоначальный датафрейм сохраняем под другим именем
data = pd.concat([x_train, y_train], axis=1)

## Задание 2. <a class='anchor' id='task02'></a>

Провести EDA (exploratory data analysis: histograms, boxplots - visualization, etc). Например

Проверим сбалансированность классов:

In [23]:
data['Revenue'].value_counts()

0    6921
1    1281
Name: Revenue, dtype: int64

* Классы несбалансированы!

Общий взгляд на датасет:

In [24]:
print ("Rows     : " ,data.shape[0])
print ("Columns  : " ,data.shape[1])
print ("\nFeatures : \n" ,data.columns.tolist())
print ("\nMissing values :  ", data.isnull().sum().values.sum()) #telcom.isnull().sum().values.sum()
print ("\nUnique values :  \n",data.nunique())

Rows     :  8202
Columns  :  18

Features : 
 ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend', 'Revenue']

Missing values :   0

Unique values :  
 Administrative               23
Administrative_Duration    2441
Informational                16
Informational_Duration      938
ProductRelated              276
ProductRelated_Duration    6637
BounceRates                1409
ExitRates                  3417
PageValues                 1819
SpecialDay                    6
Month                        10
OperatingSystems              5
Browser                      10
Region                        9
TrafficType                  14
VisitorType                   3
Weekend                       2
Revenue                       2
dtype: int64


Проверка на пустые ячейки, пробелы в строковых данных

In [25]:
all_cols = data.columns.tolist() # все столбцы сохраняем в список
counter = 0
for column in all_cols:
    indexes = data[data[column]==' '].index.tolist() # список индексов ячеек с пробелами
    indexes1 = data[data[column]==''].index.tolist()
    if (len(indexes)>0) or (len(indexes1)>0):
        item = {'column':column, 'indexes':indexes+indexes1}
        print(f'обнаружены пробелы и/или пустые ячейки: {item}')
        counter = counter +1
        
if counter == 0:
    print(f'пробелы и/или пустые ячейки не обнаружены!')

пробелы и/или пустые ячейки не обнаружены!


Проверим формат данных:

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8202 entries, 6276 to 7335
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           8202 non-null   int64  
 1   Administrative_Duration  8202 non-null   float64
 2   Informational            8202 non-null   int64  
 3   Informational_Duration   8202 non-null   float64
 4   ProductRelated           8202 non-null   int64  
 5   ProductRelated_Duration  8202 non-null   float64
 6   BounceRates              8202 non-null   float64
 7   ExitRates                8202 non-null   float64
 8   PageValues               8202 non-null   float64
 9   SpecialDay               8202 non-null   float64
 10  Month                    8202 non-null   object 
 11  OperatingSystems         8202 non-null   int64  
 12  Browser                  8202 non-null   int64  
 13  Region                   8202 non-null   int64  
 14  TrafficType          

* все данные находятся в форматах, подходящих для дальнейшей работы

Графическое отображение несбалансированности классов:

In [27]:
#labels
lab = data["Revenue"].value_counts().keys().tolist()
#values
val = data["Revenue"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data1 = [trace]
fig = go.Figure(data = data1,layout = layout)
py.iplot(fig)

### Data manipulation

#### определим количественные признаки и категориальные

In [28]:
# наименование столбца с таргетами
target_col = [data['Revenue'].name]

# категориальные столбцы:
num_cols = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration','ProductRelated',
            'ProductRelated_Duration','BounceRates','ExitRates', 'PageValues']
cat_cols = [col_name for col_name in data.columns.tolist() if (col_name not in num_cols+target_col) ]

#### Разделим датасет на две части по условию принадлежности к меткам классов

In [29]:
data_1=data[data['Revenue']==True]
data_0=data[data['Revenue']==False]

In [30]:
#function  for pie plot for customer attrition types
def plot_pie(data_1, data_0, column) :
    
    trace1 = go.Pie(values  = data_1[column].value_counts().values.tolist(),
                    labels  = data_1[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "target 1",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = data_0[column].value_counts().values.tolist(),
                    labels  = data_0[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "target 0" 
                   )


    layout = go.Layout(dict(title = column + " distribution",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "target 1(yes)",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "target 0 (no)",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    py.iplot(fig)


#function  for histogram for customer attrition types
def histogram(data_1, data_0, column) :
    trace1 = go.Histogram(x  = data_1[column],
                          histnorm= "percent",
                          name = "target 1",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = data_0[column],
                          histnorm = "percent",
                          name = "target 0 customers",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data = [trace1,trace2]
    layout = go.Layout(dict(title =column + " distribution ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data,layout=layout)
    
    py.iplot(fig)
    
#function  for scatter plot matrix  for numerical columns in data
def scatter_matrix(df):
    
    df  = df.sort_values(by = "Revenue" ,ascending = True)
    classes = df["Revenue"].unique().tolist()
    classes
    
    class_code  = {classes[k] : k for k in range(2)}
    class_code

    color_vals = [class_code[cl] for cl in df["Revenue"]]
    color_vals

    pl_colorscale = "Portland"

    pl_colorscale

    text = [df.loc[k,"Revenue"] for k in range(len(df))]
    text

    trace = go.Splom(dimensions = [dict(label  = "BounceRates",
                                       values = df["BounceRates"]),
                                  dict(label  = 'ExitRates',
                                       values = df['ExitRates']),
                                  dict(label  = 'PageValues',
                                       values = df['PageValues'])],
                     text = text,
                     marker = dict(color = color_vals,
                                   colorscale = pl_colorscale,
                                   size = 3,
                                   showscale = False,
                                   line = dict(width = .1,
                                               color='rgb(230,230,230)'
                                              )
                                  )
                    )
    axis = dict(showline  = True,
                zeroline  = False,
                gridcolor = "#fff",
                ticklen   = 4
               )
    
    layout = go.Layout(dict(title  = 
                            "Scatter plot matrix for Numerical columns for customer attrition",
                            autosize = False,
                            height = 800,
                            width  = 800,
                            dragmode = "select",
                            hovermode = "closest",
                            plot_bgcolor  = 'rgba(240,240,240, 0.95)',
                            xaxis1 = dict(axis),
                            yaxis1 = dict(axis),
                            xaxis2 = dict(axis),
                            yaxis2 = dict(axis),
                            xaxis3 = dict(axis),
                            yaxis3 = dict(axis),
                           )
                      )
    data   = [trace]
    fig = go.Figure(data = data,layout = layout )
    py.iplot(fig)

In [31]:
#for all categorical columns plot pie
for i in cat_cols:
    plot_pie(data_1, data_0, i)

Есть признак, в котором есть какое-то исчезающе малое кол-во одного значения:

In [32]:
(data['VisitorType']=='Other').value_counts()

False    8139
True       63
Name: VisitorType, dtype: int64

Посмотрим на количественные признаки:

In [33]:
#for all numerical columns plot histogram    
for i in num_cols :
    histogram(data_1, data_0, i)

## Задание 3 <a class='anchor' id='task03'></a>

Сделать feature engineering. При этом постарайтесь «завернуть» ваш код для преобразования данных в классы или функции (пример Transformer как в ноутбуке).

### Preprocess and feature engineering

### sklearn Transformers

In [34]:
from sklearn.base import BaseEstimator, TransformerMixin

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    трансформер для выбора одного столбца из датафрейма для того, чтобы сделать над ним изменения
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Трансформер для того, чтобы выбрать один столбец и произвести над ним преобразования - для
    количественных данных
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [35]:
num_cols, cat_cols
all_cols = cat_cols + num_cols
all_cols

['SpecialDay',
 'Month',
 'OperatingSystems',
 'Browser',
 'Region',
 'TrafficType',
 'VisitorType',
 'Weekend',
 'Administrative',
 'Administrative_Duration',
 'Informational',
 'Informational_Duration',
 'ProductRelated',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues']

In [36]:
# создаём новые признаки с помощью пайплайнов
from sklearn.pipeline import Pipeline

new_columns = []
for cat_feature in cat_cols:
    print(cat_feature)
    feature_object = Pipeline([
                ('selector', ColumnSelector(key=cat_feature)),
                ('ohe', OHEEncoder(key=cat_feature))
            ])
    new_columns.append(feature_object)

SpecialDay
Month
OperatingSystems
Browser
Region
TrafficType
VisitorType
Weekend


In [37]:
#new_columns[0]
new_columns[0].fit(x_test)
new_columns[0].transform(x_test)

,SpecialDay_0.0,SpecialDay_0.2,SpecialDay_0.4,SpecialDay_0.6,SpecialDay_0.8,SpecialDay_1.0
10109,1,0,0,0,0,0
10030,1,0,0,0,0,0
3236,1,0,0,0,0,0
9793,1,0,0,0,0,0
9000,1,0,0,0,0,0
...,...,...,...,...,...,...
9986,1,0,0,0,0,0
847,1,0,0,0,0,0
2955,1,0,0,0,0,0
7284,1,0,0,0,0,0


In [38]:
from sklearn.preprocessing import StandardScaler

new_columns2 = []
for num_feature in num_cols:
    print(num_feature)
    feature_object =  Pipeline([
                ('selector', NumberSelector(key=num_feature)),
                ('standard', StandardScaler())
            ])
    new_columns2.append(feature_object)

Administrative
Administrative_Duration
Informational
Informational_Duration
ProductRelated
ProductRelated_Duration
BounceRates
ExitRates
PageValues


In [39]:
new_feats = new_columns+new_columns2
new_feats[:2]

[Pipeline(steps=[('selector', ColumnSelector(key='SpecialDay')),
                 ('ohe', OHEEncoder(key='SpecialDay'))]),
 Pipeline(steps=[('selector', ColumnSelector(key='Month')),
                 ('ohe', OHEEncoder(key='Month'))])]

In [40]:
combined_feats = list(zip(all_cols, new_feats))
combined_feats[:2]

[('SpecialDay', Pipeline(steps=[('selector', ColumnSelector(key='SpecialDay')),
                  ('ohe', OHEEncoder(key='SpecialDay'))])),
 ('Month', Pipeline(steps=[('selector', ColumnSelector(key='Month')),
                  ('ohe', OHEEncoder(key='Month'))]))]

In [41]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(combined_feats)

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(x_train)

array([[ 1.        ,  0.        ,  0.        , ..., -0.45882445,
        -0.73429252,  4.58046369],
       [ 1.        ,  0.        ,  0.        , ...,  0.92111927,
         0.02951537, -0.31557279],
       [ 0.        ,  0.        ,  1.        , ..., -0.35930928,
        -0.56978005, -0.31557279],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.36914178,
         0.3503147 , -0.31557279],
       [ 1.        ,  0.        ,  0.        , ..., -0.45882445,
        -0.19962699, -0.31557279],
       [ 1.        ,  0.        ,  0.        , ..., -0.33337502,
        -0.34544486, -0.31557279]])

## Задание 4 <a class='anchor' id='task04'></a>

Обучить базовую модель (любую - на ваш выбор). При этом нужно использовать cross validation

Чтобы выполнить кросс-валидацию, вернёмся к нашему датасету, и снова пройдём весь цикл от начала до конца. В качестве эмуляции кросс-валидации буду использовать выборку из общего датасета на случайной основе


In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
av_roc = 0
roc_sum = 0
for x in range(10): # создадим 10 разных выборок
    x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=target), data[target], test_size=0.33, random_state=randint(0,99))
    test_pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = randint(0,99))),])

    test_pipeline.fit(x_train, y_train)
    preds = test_pipeline.predict(x_test)
    roc_auc = np.mean(preds == y_test)
    roc_sum = roc_sum + roc_auc
    av_roc = roc_sum/(x+1)
    item ={'roc_auc':roc_auc, 'roc_sum':roc_sum, 'av_roc':av_roc}
    print(item)

{'roc_auc': 0.8943479867011451, 'roc_sum': 0.8943479867011451, 'av_roc': 0.8943479867011451}
{'roc_auc': 0.8980421130402659, 'roc_sum': 1.792390099741411, 'av_roc': 0.8961950498707055}
{'roc_auc': 0.9009974141115626, 'roc_sum': 2.693387513852974, 'av_roc': 0.8977958379509913}
{'roc_auc': 0.9006280014776505, 'roc_sum': 3.5940155153306246, 'av_roc': 0.8985038788326561}
{'roc_auc': 0.9002585888437384, 'roc_sum': 4.494274104174363, 'av_roc': 0.8988548208348727}
{'roc_auc': 0.9039527151828592, 'roc_sum': 5.398226819357222, 'av_roc': 0.8997044698928703}
{'roc_auc': 0.8991503509420022, 'roc_sum': 6.2973771702992245, 'av_roc': 0.8996253100427464}
{'roc_auc': 0.902844477281123, 'roc_sum': 7.2002216475803476, 'av_roc': 0.9000277059475434}
{'roc_auc': 0.8980421130402659, 'roc_sum': 8.098263760620613, 'av_roc': 0.8998070845134014}
{'roc_auc': 0.902844477281123, 'roc_sum': 9.001108237901736, 'av_roc': 0.9001108237901736}


## Задание 5

